# 基于LLM微调的数学推理任务

In [1]:
import os 
from tqdm import tqdm
import re
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import Dataset
import torch
from torch.utils.data import DataLoader
from torch.nn.functional import one_hot
from sklearn.metrics import accuracy_score, f1_score
from evaluate import load
from datasets import load_dataset

os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

2024-12-10 18:35:31.212927: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-10 18:35:31.508406: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-10 18:35:32.472976: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# 设置代理
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [16]:
from modelscope.msdatasets import MsDataset
dataset =  MsDataset.load('modelscope/gsm8k', subset_name='main')

In [12]:
def get_dataloader(prompt, batch_size=1):
    def preprocess(dataset, shuffle=True):
        def collate_fn(batch):
            questions =  [[{"role": "user", "content": prompt.format(text=item['question'])}] for item in batch]
            answers = [item["answer"] for item in batch]
            return questions, answers
        return DataLoader(
            dataset,
            shuffle=shuffle,
            batch_size=batch_size,
            collate_fn=collate_fn,
        )
    testset = dataset['test']
    return preprocess(testset, shuffle=False)

In [11]:
ANS_RE = re.compile(r"#### (\-?[0-9\.\,]+)")
INVALID_ANS = "[invalid]"

def extract_answer(answer):
    match = ANS_RE.search(answer)
    if match:
        match_str = match.group(1).strip()
        match_str = match_str.replace(",", "")
        return match_str
    else:
        return INVALID_ANS
    
def is_correct(reply, truth):
    answer = extract_answer(truth)
    assert answer != INVALID_ANS
    return extract_answer(reply) == answer

In [10]:
from modelscope import AutoModelForCausalLM, AutoTokenizer

def eval_process(model_name, tokenizer_name, prompt, batch_size=1, ratio=1):
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype="auto",
        device_map="auto",
        cache_dir = '/autodl-tmp/cache'
    )
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    testloader = get_dataloader(prompt, batch_size)
    answers = []
    replies = []
    # 以下过程参照huggingface上的qwen2.5模型示例
    for idx, (texts, truths) in tqdm(enumerate(testloader), total=int(len(testloader) * ratio)):
        if idx >= int(len(testloader) * ratio):
            break
        texts = [tokenizer.apply_chat_template(
            text,
            tokenize=False,
            add_generation_prompt=True
        ) for text in texts]
        model_inputs = tokenizer(texts, return_tensors="pt", padding=True, padding_side='left').to(model.device)

        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=256
        )

        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        responses = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        replies.extend(responses)
        answers.extend(truths)
    return replies,answers
    


In [ ]:
!pwd
model_name = "./output_argu_2/qwen2_5-0_5b-instruct/v0-20241210-183550/checkpoint-1036"
tokenizer_name = "Qwen/Qwen2.5-0.5B-Instruct"

prompt = '''You are a highly skilled mathematician capable of solving complex grade-school math problems step by step. Please read the problem carefully and solve it with clear reasoning. Follow these instructions:

- Identify the known information and the question being asked.
- Break the solution into logical steps, providing clear explanations for each.
- Show all calculations and intermediate results.
- Conclude with a final answer.

Output your response in the following format:
[Explanation and calculations]
#### [Final numerical answer]  

Here is the problem:
{text}'''
replies,answers=eval_process(model_name, tokenizer_name, prompt, batch_size=8, ratio=1)


/mnt/workspace/NLP_PJ-main/code


[INFO:modelscope] Creating symbolic link /mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2___5-0___5B-Instruct -> /mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2.5-0.5B-Instruct.
[WARNING:modelscope] Failed to create symbolic link /mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2___5-0___5B-Instruct -> /mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2.5-0.5B-Instruct: [Errno 17] File exists: '/mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2___5-0___5B-Instruct' -> '/mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2.5-0.5B-Instruct'
 96%|█████████▌| 158/165 [19:21<00:51,  7.41s/it]

In [6]:
import json
total = 0
correct = 0
for reply, answer in zip(replies, answers):
    total += 1
    correct += is_correct(reply, answer)
    
print(f"正确率: {(correct / total) * 100: .2f}%")
with open('output.jsonl', 'w', encoding='utf-8') as f:
    for reply, answer in zip(replies, answers):
        # 创建一个字典
        data = {
            "reply": reply,
            "answer": answer
        }
        # 将字典转换为 JSON 格式，并写入文件
        json.dump(data, f, ensure_ascii=False)
        f.write('\n')

NameError: name 'replies' is not defined

In [7]:
# 导入swift框架进行微调
from swift.llm import (
    DatasetName, InferArguments, ModelType, SftArguments,
    infer_main, sft_main, app_ui_main
)

[INFO:swift] Successfully registered `/usr/local/lib/python3.10/site-packages/swift/llm/data/dataset_info.json`


In [8]:
model_type = ModelType.qwen2_5_0_5b_instruct
sft_args = SftArguments(
    model_type=model_type,
    dataset=['gsm8k_argu.jsonl'],
    output_dir='output_argu_2',
    max_length=4096)
result = sft_main(sft_args)
last_model_checkpoint = result['last_model_checkpoint']

[INFO:swift] Setting template_type: qwen2_5
[INFO:swift] Setting args.lazy_tokenize: False
[INFO:swift] Setting args.dataloader_num_workers: 1
[INFO:swift] output_dir: /mnt/workspace/NLP_PJ-main/code/output_argu_2/qwen2_5-0_5b-instruct/v0-20241210-183550
[INFO:swift] Start time of running main: 2024-12-10 18:35:50.307628
[INFO:swift] args: SftArguments(model_type='qwen2_5-0_5b-instruct', model_id_or_path='qwen/Qwen2.5-0.5B-Instruct', model_revision='master', full_determinism=False, sft_type='lora', freeze_parameters=[], freeze_vit=False, freeze_parameters_ratio=0.0, additional_trainable_parameters=[], tuner_backend='peft', template_type='qwen2_5', output_dir='/mnt/workspace/NLP_PJ-main/code/output_argu_2/qwen2_5-0_5b-instruct/v0-20241210-183550', add_output_dir_suffix=True, ddp_backend=None, ddp_find_unused_parameters=None, ddp_broadcast_buffers=None, ddp_timeout=1800, seed=42, resume_from_checkpoint=None, resume_only_model=False, ignore_data_skip=False, dtype='bf16', packing=False, tr

device_count: 1
rank: -1, local_rank: -1, world_size: 1, local_world_size: 1


[WARNING:modelscope] Using branch: master as version is unstable, use with caution
[INFO:modelscope] Creating symbolic link /mnt/workspace/.cache/modelscope/hub/qwen/Qwen2___5-0___5B-Instruct -> /mnt/workspace/.cache/modelscope/hub/qwen/Qwen2.5-0.5B-Instruct.
[WARNING:modelscope] Failed to create symbolic link /mnt/workspace/.cache/modelscope/hub/qwen/Qwen2___5-0___5B-Instruct -> /mnt/workspace/.cache/modelscope/hub/qwen/Qwen2.5-0.5B-Instruct: [Errno 17] File exists: '/mnt/workspace/.cache/modelscope/hub/qwen/Qwen2___5-0___5B-Instruct' -> '/mnt/workspace/.cache/modelscope/hub/qwen/Qwen2.5-0.5B-Instruct'
[INFO:swift] Loading the model using model_dir: /mnt/workspace/.cache/modelscope/hub/qwen/Qwen2___5-0___5B-Instruct
[INFO:swift] model_kwargs: {'device_map': 'cuda:0'}
[INFO:swift] model.max_model_len: 32768
[INFO:swift] model.hf_device_map: {'': device(type='cuda', index=0)}
[INFO:swift] model_config: Qwen2Config {
  "_attn_implementation_autoset": true,
  "_name_or_path": "/mnt/worksp

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/16744 [00:00<?, ? examples/s]

...

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


{'eval_loss': 0.25266954, 'eval_acc': 0.9134709, 'eval_runtime': 6.2557, 'eval_samples_per_second': 26.695, 'eval_steps_per_second': 26.695, 'epoch': 1.0, 'global_step/max_steps': '1036/1036', 'percentage': '100.00%', 'elapsed_time': '40m 37s', 'remaining_time': '0s'}


[INFO:swift] Saving model checkpoint to /mnt/workspace/NLP_PJ-main/code/output_argu_2/qwen2_5-0_5b-instruct/v0-20241210-183550/checkpoint-1036
[INFO:swift] last_model_checkpoint: /mnt/workspace/NLP_PJ-main/code/output_argu_2/qwen2_5-0_5b-instruct/v0-20241210-183550/checkpoint-1036
[INFO:swift] best_model_checkpoint: /mnt/workspace/NLP_PJ-main/code/output_argu_2/qwen2_5-0_5b-instruct/v0-20241210-183550/checkpoint-1036
[INFO:swift] images_dir: /mnt/workspace/NLP_PJ-main/code/output_argu_2/qwen2_5-0_5b-instruct/v0-20241210-183550/images


{'train_runtime': 2438.1581, 'train_samples_per_second': 6.799, 'train_steps_per_second': 0.425, 'train_loss': 0.27689182, 'epoch': 1.0, 'global_step/max_steps': '1036/1036', 'percentage': '100.00%', 'elapsed_time': '40m 38s', 'remaining_time': '0s'}


[INFO:swift] End time of running main: 2024-12-10 19:17:01.835062
